# Algoritmo de agrupamiento para la segmentación de tiendas

## Contexto

En el mundo competitivo de hoy, es crucial comprender el comportamiento de las tiendas y categorizar a los clientes según su demografía y comportamiento de compra.

## Objetivo

Este proyecto demuestra el concepto de segmentación de un conjunto de datos que pertenecen a tiendas Walmart con el producto café utilizando agrupación de k-means en Python. El conjunto de datos contiene las ventas mensuales de los años 2019 y 2020 de 2411 tiendas así como otras características. Usaremos el algoritmo de agrupación de k-means para derivar la cantidad óptima de agrupaciones y comprender los segmentos de clientes subyacentes en función de los datos proporcionados.

## Acerca del conjunto de datos

El conjunto de datos consta de las ventas mensuales de los años 2019 y 2020 de 2411 tiendas. 

### Exploración de los datos usando bibliotecas numpy y pandas en python.

In [2]:
# Cargar los paquetes necesarios
import numpy as np
import pandas as pd
import bamboolib
import pdpipe as pdp 
from tqdm import tqdm

# Importar estilo de trama
import matplotlib as plt
import seaborn as sns; sns.set()  # for plot styling
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

In [3]:
# Funciones
def log_shape(df):
    """
    Tamaño del data frame.
    """
    print(f'shape = {df.shape}')
    return df

In [4]:
# Lee el archivo csv
dataset = pd.read_csv('Cafes_Walmart.csv', index_col=None)
# Explora el conjunto de datos
log_shape(dataset)

shape = (260780, 62)


,Store Nbr,Store Name,Tramos Café,Tramos Lacteos,Tramos Baby Food,Tramos Cremadores,Tramos Leche Polvo,Tramos Cereales,Tramos Leche Liquida,Tramos Formulas Infantiles,...,Línea,Material Goup,Peso PZA,Marca,División,NSE,# Habitantes,Formato,SOM,Center Point
0,58,BA SANTA MONICA,9,10,10,4,1,6,9,1,...,Café Soluble,Nescafé Tasters Choice,0.100,Nescafé Tasters Choice,F&B,4,496,Bodega,0,0
1,58,BA SANTA MONICA,9,10,10,4,1,6,9,1,...,Café Soluble,Nescafé Tasters Choice,0.100,Nescafé Tasters Choice,F&B,4,496,Bodega,0,0
2,58,BA SANTA MONICA,9,10,10,4,1,6,9,1,...,Café Soluble,Nescafé Tasters Choice,0.100,Nescafé Tasters Choice Descafeinado,F&B,4,496,Bodega,0,0
3,58,BA SANTA MONICA,9,10,10,4,1,6,9,1,...,Café Soluble,Nescafé Tasters Choice,0.100,Nescafé Tasters Choice Descafeinado,F&B,4,496,Bodega,0,0
4,58,BA SANTA MONICA,9,10,10,4,1,6,9,1,...,Café Soluble,Nescafé Tasters Choice,0.048,Nescafé Tasters Choice,F&B,4,496,Bodega,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260775,5855,SC LAS ALAMEDAS,9,4,3,4,5,6,3,10,...,Dolce Gusto,Nescafé Dolce Gusto Cápsulas,0.189,Nescafé Dolce Gusto Cápsulas,F&B,4,1383,Hiper,0,0
260776,5855,SC LAS ALAMEDAS,9,4,3,4,5,6,3,10,...,Dolce Gusto,Nescafé Dolce Gusto Cápsulas,0.140,Nescafé Dolce Gusto Cápsulas,F&B,4,1383,Hiper,0,0
260777,5855,SC LAS ALAMEDAS,9,4,3,4,5,6,3,10,...,Dolce Gusto,Nescafé Dolce Gusto Cápsulas,0.140,Nescafé Dolce Gusto Cápsulas,F&B,4,1383,Hiper,0,0
260778,5855,SC LAS ALAMEDAS,9,4,3,4,5,6,3,10,...,PREMIUM,Nescafé Tasters Choice,0.192,Nescafé Tasters Choice,F&B,4,1383,Hiper,0,0


In [5]:
# Se calcula las sumas de las ventas por año. 
# Se han eliminado los tramos, así como Región Nescudero porque algunos registros presentan el caracter -
# Store Nbr presenta el mismo número de indentificadores que Store Name, por lo tanto se elimina.
# Expr1049 contiene lo mismo que Canal, por lo tanto se elimina.
# Material Group es igual que Marca, también se elimina.
df_ = dataset.drop(columns=['Tramos Café', 'Tramos Lacteos', 'Tramos Baby Food', 'Tramos Cremadores', 'Tramos Leche Polvo', 'Tramos Cereales', 'Tramos Leche Liquida', 'Tramos Formulas Infantiles', 'Tramos Orgánicos', 'Tramos A', 'Tramos B', 'Tramos C', 'Store Nbr', 'Región Nescuadron', 'Expr1049', 'Material Goup'])
df_["Total_2019"] = df_['2019/01']+df_['2019/02']+df_['2019/03']+df_['2019/04']+df_['2019/05']+df_['2019/06']+df_['2019/07']+df_['2019/08']+df_['2019/09']+df_['2019/10']+df_['2019/11']+df_['2019/12']
df_["Total_2020"] = df_['2020/01']+df_['2020/02']+df_['2020/03']+df_['2020/04']+df_['2020/05']+df_['2020/06']+df_['2020/07']+df_['2020/08']+df_['2020/09']
df_ = df_.drop(columns=['2019/01', '2019/02', '2019/03', '2019/04', '2019/05', '2019/06', '2019/07', '2019/08', '2019/09', '2019/10', '2019/11', '2019/12', '2020/01', '2020/02', '2020/03', '2020/04', '2020/05', '2020/06', '2020/07', '2020/08', '2020/09'])
df_ = df_.reset_index().drop(columns = 'index')

In [6]:
# Se renombran algunas características, sólo se quitan acentos y espacios.
df_ = df_.rename(columns={'Región Nestlé': 'Region_Nestle', 'Data Type': 'Data_Type', 'Clasificación Store Optimizer': 'Clasificacion_Store_Optimizer', 'Categoría': 'Categoria', 'División': 'Division', 'Store Name': 'Store Name', 'Center Point': 'Center_Point', 'Línea': 'Linea'})
log_shape(df_)
sample = df_

shape = (260780, 27)


In [90]:
print('Seleccione los identificadores de interés para agrupar\n1. Data Type\n2. Región Nestlé\n3. Clasificación Store Optimizer\n4. NSE\n5. SOM')
parametros = []
a = True
print('\nEscribe el número correspondiente al campo. \nCuando desee terminar, escribe la palabra Fin.\n')
while a == True:
    x = input('Escribe un número:\n')
    if x == ('Fin'):
        a = False
    else:
        parametros.append(x)

print(f'Seleccionaste {parametros}')

campo = []
clases = []

for count in parametros: 
    if count == '1':
        print('Data Type')
        campo.append('Data_Type')
        print('\nElige:\n1. POS Sales\n2. POS Qty')
        x = input()
        if x == '1':
            print('Seleccionaste: POS Sales')
            clases.append('POS Sales')
        elif x == '2':
            print('Seleccionaste: Pos Qty')
            clases.append('POS Qty')
        else:
            print('Error')
            
    elif count == '2':
        print('\nRegión Nestlé')
        campo.append('Region_Nestle')
        print('\nElige:\n1. Valle de México\n2. Occidente\n3. Norte\n4. Anahuac\n5. Sureste\n6. Pacífico')
        x = input()
        if x == '1':
            print('Seleccionaste: Valle de Mexico')
            clases.append('Valle de Mexico')
        elif x == '2':
            print('Seleccionaste: Occidente')
            clases.append('Occidente')
        elif x == '3':
            print('Seleccionaste: Norte')
            clases.append('Norte')
        elif x == '4':
            print('Seleccionaste: Anahuac')
            clases.append('Anahuac')
        elif x == '5':
            print('Seleccionaste: Sureste')
            clases.append('Sureste')
        elif x == 6:
            print('Seleccionaste: Pacífico')
            clases.append('Pacífico')
        else:
            print('Error')
            
    elif count == '3':
        print('\nClasificación Store Optimizer')
        campo.append('Clasificacion_Store_Optimizer')
        print('\nElige:\n1. Oro\n2. Platino\n3. Plata\n4. Resto')
        x = input()
        if x == '1':
            print('Seleccionaste: Oro')
            clases.append('Oro')
        elif x == '2':
            print('Seleccionaste: Platino')
            clases.append('Platino')
        elif x == '3':
            print('Seleccionaste: Plata')
            clases.append('Plata')
        elif x == '4':
            print('Seleccionaste: Resto')
            clases.append('Resto')
        else:
            print('Error')
            
    elif count == '4':
        print('\nNSE')
        campo.append('NSE')
        print('\nElige:\n1. 1\n2. 2\n3. 3\n4. 4\n5. 5')
        x = input()
        if x == '1':
            print('Seleccionaste: 1')
            clases.append(1)
        elif x == '2':
            print('Seleccionaste: 2')
            clases.append(2)
        elif x == '3':
            print('Seleccionaste: 3')
            clases.append(3)
        elif x == '4':
            print('Seleccionaste: 4')
            clases.append(4)
        elif x == '5':
            print('Seleccionaste: 5')
            clases.append(5)
        else:
            print('Error')
            
    elif count == '5':
        print('\nSOM')
        campo.append('SOM')
        print('\nElige:\n0. 0\n1. 1')
        x = input()
        if x == '0':
            print('Seleccionaste: 0')
            clases.append(0)
        elif x == '1':
            print('Seleccionaste: 1')
            clases.append(1)
        else:
            print('Error')
            
    elif count == '6':
        print('\nCenter Point')
        campo.append('Center_Point')
        print('\nElige:\n0. 0\n1. 1')
        x = input()
        if x == '0':
            print('Seleccionaste: 0')
            clases.append(0)
        elif x == '1':
            print('Seleccionaste: 1')
            clases.append(1)
        else:
            print('Error')
            
    else:
        print('Error')


mi_df = sample.set_index(campo)
mi_df = mi_df.loc[[clases]]
mi_df = mi_df.reset_index()
mi_df

Seleccione los identificadores de interés para agrupar
1. Data Type
2. Región Nestlé
3. Clasificación Store Optimizer
4. NSE
5. SOM

Escribe el número correspondiente al campo. 
Cuando desee terminar, escribe la palabra Fin.

Escribe un número:
2
Escribe un número:
4
Escribe un número:
Fin
Seleccionaste ['2', '4']

Región Nestlé

Elige:
1. Valle de México
2. Occidente
3. Norte
4. Anahuac
5. Sureste
6. Pacífico
5
Seleccionaste: Sureste

NSE

Elige:
1. 1
2. 2
3. 3
4. 4
5. 5
1
Seleccionaste: 1


,Region_Nestle,NSE,Store Name,Exhibiciones,UPC walmart,UPC,Data_Type,Estado,Clasificacion_Store_Optimizer,Longitud,...,Linea,Peso PZA,Marca,Division,# Habitantes,Formato,SOM,Center_Point,Total_2019,Total_2020
0,Sureste,1,SC PLAYA DEL CARMEN,23,761303287065,7613032870652,POS Sales,Quintana Roo,Platino,20.629747,...,Dolce Gusto,0.188,Nescafé Dolce Gusto Cápsulas,F&B,1377,Hiper,1,1,14485.45,3699.30
1,Sureste,1,SC PLAYA DEL CARMEN,23,761303287065,7613032870652,POS Qty,Quintana Roo,Platino,20.629747,...,Dolce Gusto,0.188,Nescafé Dolce Gusto Cápsulas,F&B,1377,Hiper,1,1,127.00,29.00
2,Sureste,1,SC PLAYA DEL CARMEN,23,761303287065,7613032870652,POS Sales,Quintana Roo,Platino,20.629747,...,Dolce Gusto,0.188,Nescafé Dolce Gusto Cápsulas,F&B,1377,Hiper,1,1,0.00,4382.41
3,Sureste,1,SC PLAYA DEL CARMEN,23,761303287065,7613032870652,POS Qty,Quintana Roo,Platino,20.629747,...,Dolce Gusto,0.188,Nescafé Dolce Gusto Cápsulas,F&B,1377,Hiper,1,1,0.00,37.00
4,Sureste,1,SC PLAYA DEL CARMEN,23,750105927325,7501059273252,POS Sales,Quintana Roo,Platino,20.629747,...,Dolce Gusto,0.096,Nescafé Dolce Gusto Cápsulas,F&B,1377,Hiper,1,1,40582.84,1767.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,Sureste,1,BA CARDENAS,9,750105864588,7501058645883,POS Qty,Tabasco,Resto,17.987379,...,Café Soluble,0.120,Nescafé Cappuccino,F&B,781,Bodega,1,1,368.00,158.00
4976,Sureste,1,BA CARDENAS,9,750105863993,7501058639936,POS Sales,Tabasco,Resto,17.987379,...,Café Soluble,0.010,Nescafe Café de Olla,F&B,781,Bodega,1,1,3687.30,3028.00
4977,Sureste,1,BA CARDENAS,9,750105863993,7501058639936,POS Qty,Tabasco,Resto,17.987379,...,Café Soluble,0.010,Nescafe Café de Olla,F&B,781,Bodega,1,1,1286.00,1514.00
4978,Sureste,1,BA CARDENAS,9,750105864802,7501058648020,POS Sales,Tabasco,Resto,17.987379,...,Café Soluble,0.200,Nescafé Decaf,F&B,781,Bodega,1,1,82612.80,73186.08
